In [10]:
import re

In [1]:
def clean_float(val):
    value = val.strip()
    parts = value.replace(',', '.').split('.')
    clean = ''

    if parts[0].startswith('-'):
        clean += '-'

    parts[0] = parts[0].strip('+-').strip()
    parts[1] = parts[1].strip()
    
    #kui enne komakohta oli ainult - või +, siis 0.xxx
    if len(parts[0]) == 0:
        parts[0] = '0'
    #peale komakohta lisab 0: 4. ->  4.0
    if len(parts[1]) == 0:
        parts[1] = '0'

    #clean on kas - või tühi; 
    clean += parts[0].strip() + '.' + parts[1].strip()
    return clean


In [2]:
#clean_float('4.')

In [22]:
import sys
import os
import pathlib
__file__ = os.path.abspath('')
filename = 'possible_units.csv'
directory = pathlib.Path(__file__).parent.parent.as_posix()

possible_units = open("/home/anneott/Repos/cda-data-cleaning/cda_data_cleaning/shared_workflows/loinc_cleaning/development/loinc_unit_assignment/data/" + filename)
possible_units = possible_units.read()
possible_units = possible_units.split('\n')
possible_units = sorted(possible_units)

In [23]:
def remove_units(raw_text, strict):
    text = raw_text.strip()
    selected_unit = ""
    for unit in possible_units:
        if text.split()[-1] == unit and len(unit) > len(selected_unit) and strict\
                or text.endswith(unit) and len(unit) > len(selected_unit) and not strict:
            selected_unit = unit
    if selected_unit != "":
        selected_unit = selected_unit
        return text[:len(text) - len(selected_unit)].strip()
    return text

In [25]:
strict = False
raw_text = '9.8 10g/l - 10.8 10g/l'
#selected_unit = '10g/l'
#raw_text[:len(raw_text) - len(selected_unit)] #see töötab õigesti, alumine mitte
#LAHENDUS PANE POSSILBE UNITS TÄHESTIKU JRK
#remove_units(raw_text, strict)
remove_units(raw_text, strict)

'9.8 10g/l - 10.8'

In [7]:
import re
clean = []
type = []

def match_float(text):
        m1 = re.match("^\s*[+-]?[0-9][\.,][0-9]*\s*$", text)
        print('m1', m1)
        m2 = re.match("^\s*[+-]?[\.,][0-9]+\s*$", text)
        print('m2', m2)
        m3 = re.match("^\s*[+-]?[1-9][0-9]*[\.,][0-9]*\s*$", text)
        print('m3', m3)
        return m1 or m2 or m3

def is_float(text):
    units_removed = remove_units(text, True)
    print('units_removed', units_removed)
    if match_float(units_removed):
        clean.append(clean_float(units_removed))
        type.append("float")
        return True
    return False

In [16]:
#is_float('3.4 10g/l')
#is_float('4.5ab')
match_float('4.5')

m1 <_sre.SRE_Match object; span=(0, 3), match='4.5'>
m2 None
m3 <_sre.SRE_Match object; span=(0, 3), match='4.5'>


<_sre.SRE_Match object; span=(0, 3), match='4.5'>

INTEGER

In [7]:
def match_integer(text):
    m1 = re.match("^\s*[+-]?[1-9][0-9]*\s*$", text)
    m2 = re.match("^\s*0\s*$", text)
    return m1 or m2


def is_integer(text):
    units_removed = remove_units(text, True)
    if match_integer(units_removed):
        clean.append(self.clean_integer(units_removed))
        type.append("integer")
        return True
    return False

def clean_integer(val):
    value = val.strip()
    clean = ''

    if value.startswith('-'):
        clean += '-'

    clean += value.strip('+-').strip()
    return clean




NUM AND PAR

In [72]:
def match_num_and_par(text):
    parts = text.strip().split('(')
    if len(parts) == 2 and parts[1].strip().endswith(')'):
        parts[1] = parts[1].strip().strip(')')
        type_p1 = None
        type_p2 = None
        if match_integer(parts[0]):
            type_p1 = "integer"
            clean_p1 = clean_integer(parts[0])
        elif match_float(parts[0]):
            type_p1 = "float"
            clean_p1 = clean_float(parts[0])
            
        print('parts 1', parts[1])
        if match_integer(parts[1]):
            type_p2 = "integer"
            clean_p2 = clean_integer(parts[1])
        elif match_float(parts[1]):
            type_p2 = "float"
            clean_p2 = clean_float(parts[1])
        print('cleanp1', clean_p1)
        print('cleanp2', clean_p2)
        
        if type_p1 == "float" and type_p2 == "float":
            print('1')
            if clean_p1 == clean_p2:
                return ["num_and_par", clean_p1]
        elif type_p1 == "float" and type_p2 == "integer":
            print('2')
            if float(clean_p1) == float(clean_p2):
                return ["num_and_par", clean_p1]
        elif type_p1 == "integer" and type_p2 == "float":
            print('3')
            if float(clean_p1) == float(clean_p2):
                return ["num_and_par", clean_p2]
        elif type_p1 == "integer" and type_p2 == "integer":
            print('4')
            if clean_p1 == clean_p2:
                return ["num_and_par", clean_p1]
    return False


In [80]:
match_num_and_par('4(4')
#print(match_integer('4 10g/l'))

False

In [15]:
def match_between(text):
        text = text.strip().split('-')
        print('text ', text)
        val1 = None
        val2 = None
        if len(text) == 4 and text[0].strip() == '' and text[2].strip() == '':
            print('len4')
            val1 = '-' + text[1].strip()
            val2 = '-' + text[3].strip()
        if len(text) == 3 and text[0].strip() == '':
            print('len3')
            val1 = '-' + text[1].strip()
            val2 = text[2].strip()
        if len(text) == 2:
            print('len2')
            val1 = text[0].strip()
            val2 = text[1].strip()

        if val1 and val2:
            #result = Range()
            print('val1 ja val2 ', val1, val2)
            #result.min_inc = True
            #result.max_inc = True
            min_val = None
            max_val = None
            if match_integer(val1):
                min_val = clean_integer(val1)
            elif match_float(val1):
                min_val = clean_float(val1)
            if match_integer(val2):
                max_val = clean_integer(val2)
            elif match_float(val2):
                max_val = clean_float(val2)

    
            if min_val and max_val:
                result1 = min_val
                result2 = max_val
                if float(max_val) > float(min_val):
                    return [result1, result2]
        return False



In [16]:
match_between('-2--4')

text  ['', '2', '', '4']
len4
val1 ja val2  -2 -4


False

In [6]:
-4 > -2

False